# USG grants crawl
## Agency-specific replication from  Lee & Chung (2022)

### Previously

In the previous chapter we replicated some of our previous analysis, which was done with a list of open-science terms of our own creation, with an emperically derived list of terms derived from [Lee & Chung (2022)](https://doi.org/10.47989/irpaper949).  While the reults look compelling, and at least appear to plausible relatie to our intuitions, we should always be skeptical of our results until we do some thorough sanity checks.  

In this chapter we'll implement some sanity checks.  These will include confirming that the extreme ends of the term-matching distributions (e.g. many term matches vs no term matches) do indeed reflect the sorts of grants we would expect.  We'll also check to make sure that our quantative analyses of these descriptions actually return the sorts of results we would expect.

### Loading the database once more

Let's begin by loading up the database provided by the website, which is stored in an xml format.

In [1]:
# import our helper functions
import sys
import os
sys.path.insert(0, os.path.abspath('../src'))
sys.path
import grantsGov_utilities as grantsGov_utilities

# local data storage directory
localDataDir='inputData'

grantsDF=grantsGov_utilities.detectLocalGrantData(localPath='../'+localDataDir,forceDownload=True)
grantsDF

,OpportunityID,OpportunityTitle,OpportunityNumber,AgencyCode,AgencyName,AwardCeiling,AwardFloor,EstimatedTotalProgramFunding,ExpectedNumberOfAwards,Description
0,262148,Establishment of the Edmund S. Muskie Graduate...,SCAPPD-14-AW-161-SCA-08152014,DOS-SA,Bureau of South and Central Asian Affairs,600000,400000,600000,1,The Office of Press and Public Diplomacy of th...
1,262149,Eradication of Yellow Crazy Ants on Johnston A...,F14AS00402,DOI-FWS,Fish and Wildlife Service,0,0,0,NaN,Funds under this award are to be used for the ...
2,131073,"Cooperative Ecosystem Studies Unit, Piedmont S...",G12AS20003,DOI-USGS1,Geological Survey,0,0,31900,1,The USGS Southeast Ecological Science Center s...
3,131094,Plant Feedstock Genomics for Bioenergy: A Joi...,DE-FOA-0000598,PAMS-SC,Office of Science,500000,200000,6000000,10,The U.S. Department of Energy&apos;s Office of...
4,131095,Management of HIV-Related Lung Disease and Car...,RFA-HL-12-034,HHS-NIH11,National Institutes of Health,400000,NaN,2000000,NaN,This FOA invites clinical trials planning gran...
...,...,...,...,...,...,...,...,...,...,...
70325,262109,2014/2015 Social Responsibility through Englis...,RELO-BP-MOB,DOS-HUN,U.S. Mission to Hungary,152418,152418,152418,1,"In close consultation with RELO Budapest, the ..."
70326,262108,"Notice of Intent to Award - Fort McHenry, Balt...",NPS-14-NERO-0124,DOI-NPS,National Park Service,65000,65000,65000,1,United States Department of the Interior Natio...
70327,262112,Fish and Wildlife Coordination Act,R14AS00070,DOI-BOR,Bureau of Reclamation,525000,525000,525000,1,"To provide financial assistance, through grant..."
70328,131053,USAID/Uganda Literacy and Health Education Pro...,RFA-617-12-000001,USAID-UGA,Uganda USAID-Kampala,57000000,0,57000000,1,Literacy Program is a 5-year program to improv...


## Cleaning
As before, we need to do a bit of cleaning, so lets do a more comprehensive version of that here.

Note:  This may take a moment

In [2]:
grantsDF=grantsGov_utilities.prepareGrantsDF(grantsDF, repair=True)
grantsDF

OpportunityID                    int64
OpportunityTitle                object
OpportunityNumber               object
AgencyCode                      object
AgencyName                      object
AwardCeiling                     int64
AwardFloor                       int64
EstimatedTotalProgramFunding     int64
ExpectedNumberOfAwards           int64
Description                     object
dtype: object
62144 grant agency name or code value records altered


C:\Users\dbullock\Documents\code\gitDir\USG_grants_crawl\src\grantsGov_utilities.py:216: UserWarning: NOTE: this function CHANGES the values / content of the grantsDF from the information contained on grants.gov, including but not limited to adding data columns, replacing null/empty values, and/or inferring missing values.
  warn('NOTE: this function CHANGES the values / content of the grantsDF from the information contained on grants.gov, including but not limited to adding data columns, replacing null/empty values, and/or inferring missing values.')


28027 grant funding value records repaired


,OpportunityID,OpportunityTitle,OpportunityNumber,AgencyCode,AgencySubCode,AgencyName,AwardCeiling,AwardFloor,EstimatedTotalProgramFunding,ExpectedNumberOfAwards,Description
0,262148,Establishment of the Edmund S. Muskie Graduate...,SCAPPD-14-AW-161-SCA-08152014,DOS,SA,Bureau of South and Central Asian Affairs,600000,400000,600000,1,The Office of Press and Public Diplomacy of th...
1,262149,Eradication of Yellow Crazy Ants on Johnston A...,F14AS00402,DOI,FWS,Fish and Wildlife Service,0,0,0,0,Funds under this award are to be used for the ...
2,131073,"Cooperative Ecosystem Studies Unit, Piedmont S...",G12AS20003,DOI,USGS1,Geological Survey,0,0,31900,1,The USGS Southeast Ecological Science Center s...
3,131094,Plant Feedstock Genomics for Bioenergy: A Joi...,DE-FOA-0000598,PAMS,SC,Office of Science,500000,200000,6000000,10,The U.S. Department of Energy&apos;s Office of...
4,131095,Management of HIV-Related Lung Disease and Car...,RFA-HL-12-034,HHS,NIH11,National Institutes of Health,400000,0,2000000,0,This FOA invites clinical trials planning gran...
...,...,...,...,...,...,...,...,...,...,...,...
70325,262109,2014/2015 Social Responsibility through Englis...,RELO-BP-MOB,DOS,HUN,U.S. Mission to Hungary,152418,152418,152418,1,"In close consultation with RELO Budapest, the ..."
70326,262108,"Notice of Intent to Award - Fort McHenry, Balt...",NPS-14-NERO-0124,DOI,NPS,National Park Service,65000,65000,65000,1,United States Department of the Interior Natio...
70327,262112,Fish and Wildlife Coordination Act,R14AS00070,DOI,BOR,Bureau of Reclamation,525000,525000,525000,1,"To provide financial assistance, through grant..."
70328,131053,USAID/Uganda Literacy and Health Education Pro...,RFA-617-12-000001,USAID,UGA,Uganda USAID-Kampala,57000000,0,57000000,1,Literacy Program is a 5-year program to improv...


### Keywords and terms

Although we aren't going to inspect the keywords and agencies on their own this time, we still need to collect them.  Once we have loaded them, we can determine which words are occuring in which grants.  The resulting information can be placed in a dictionary, where the relevant information can be accessed by using the [tuple](https://www.w3schools.com/python/python_tuples.asp) corresponding to the desired agency and keyword (e.g. ('agency','keyword')).  From this data structure, we can compute the frequency with which grant IDs are appearing, select the top three, and three random grants that do not appear in the listing.

In [5]:
import json
import seaborn as sns
import itertools
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np

#HERE'S THE CHANGE FROM THE PREVOUS NOTEBOOK
# open the keywords csv file
inputKeywords=pd.read_csv('../OSterms_LeeChung2022.csv')

# split it into a list.  Each term is kept on a separate line
keywords=inputKeywords['terms'].tolist()

# find the grants that are associated with these keywords
grantFindsOut=grantsGov_utilities.searchGrantsDF_for_keywords(grantsDF,keywords)

# get a count of the counts for the unique grant IDs
allFoundGrantIDConcatList=[grantID for keywordMatches in grantFindsOut.values() for grantID in keywordMatches]
grantIDs,counts=np.unique(allFoundGrantIDConcatList, return_counts=True)
topIndexes=np.argsort(counts)
print(grantIDs[topIndexes[-5:-1]])
print(counts[topIndexes[-5:-1]])

[1]


In [29]:
grantIDs[topIndexes[-1]]

314589

### Viewing the top match

Now what we have the match counts and their indexes, lets look at the description of the grant that has the most matches.

In [35]:
#set up a clean text printing method
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

# get the description match for the current text
topMatchText=grantsDF['Description'].loc[grantsDF['OpportunityID'].eq(grantIDs[topIndexes[-1]])].values[0]
# bold the words that match
for iKeyword in keywords:

    topMatchText=topMatchText.replace(iKeyword,'**'+iKeyword+'**')
# display the text as markdown
printmd(topMatchText)

the idea development award is intended to support new ideas that represent innovative approaches to prostate cancer **research** and have the potential to make an important contribution to the pcrp mission. the key components of this award mechanism are: **innovation**: **research** deemed innovative may represent a new paradigm, challenge current paradigms, look at existing problems from new perspectives, leverage unique study populations, or exhibit other highly creative qualities. **research** that is an incremental advance upon published data is not considered innovative. multidisciplinary projects are especially encouraged. impact: applications are required to address and provide a solution to one or more of the fy19 pcrp overarching challenges. the potential impact of the **research**, both shortterm and longterm, in addressing the fy19 pcrp overarching challenge(s) should be clearly described. highimpact **research** will, if successful, significantly advance prostate cancer **research** and/or patient care. preliminary data: due to this award&#8217;s emphasis on **innovation**, the presentation of preliminary data relevant to prostate cancer and the proposed project is encouraged, but not required. any unpublished, preliminary data provided should originate from the laboratory of the principal investigator (pi) or a member(s) of the **research** team. regardless whether preliminary data is included or not, applications should be based on a sound scientific rationale that is established through logical reasoning and/or critical review and analysis of the literature. to maximize the potential for impact, investigators are strongly encouraged to incorporate the following components into their study design where appropriate: authentication of proposed cell lines; statistical rigor of preclinical animal experiments and epidemiological studies; incorporation of experiments to assess clinical relevance and translatability of findings; and validation in patient cohorts. as such, the pcrpfunded prostate cancer biorepository network (pcbn) (http://www.prostatebiorepository.org) and/or the north carolina &#8211; louisiana prostate cancer project (pcap) (https://pcap.bioinf.unc.edu) are important resources to consider if retrospectively collected human anatomical substances or correlated clinical data are critical to the proposed studies. studies utilizing data derived from large patient studies that include longterm health records, biospecimen **repositories**, and preexisting **research** and that apply stateoftheart genomic and/or proteomic analysis, **bioinformatics**, and/or mathematical models to such data are also encouraged. new investigator category: the idea development award mechanism encourages **research** ideas from investigators in the early stages of their careers. the new investigator category of this award mechanism is designed to allow applications naming pis early in their faculty appointments, or in the process of developing independent **research** careers, to compete for funding separately from established investigators. all new investigator applicants must meet specific eligibility criteria as described in section ii.c, eligibility information. pis using the new investigator category are required to include a collaborator (or collaborators) appropriate to the application who has experience in prostate cancer **research** as demonstrated by a record of funding and publications in prostate cancer **research**. it is the responsibility of the pi to describe how the **collaboration**(s) will augment his or her expertise to best address the **research** question. multidisciplinary projects are encouraged, and multiinstitutional projects are allowed. each proposed study must include a clearly stated plan for interactions among all team members and organizations involved. the plan must include communication, coordination of **research** progress and results, and data transfer. additionally, multiinstitutional applications must provide an **intellectual property** plan to resolve potential intellectual and material property issues and to remove institutional barriers that might interfere with achieving high levels of cooperation to ensure the successful completion of this award. **research** involving human subjects is permitted under this funding opportunity; however clinical trials are not allowed under this funding opportunity. correlative studies associated with an existing clinical trial are particularly encouraged, provided they are determined to be no greater than minimal risk by the institutional review board (irb) of record and the u.s. army medical **research** and materiel command (usamrmc) office of **research** protections (orp), human **research** protection office (hrpo). pis seeking funding for a clinical trial are encouraged to consider submitting an application to the fy19 pcrp clinical trial award (funding opportunity number: w81xwh19pcrpcta). a congressionally mandated metastatic cancer task force was formed with the purpose of identifying ways to help accelerate clinical and translational **research** aimed at extending the lives of advanced state and recurrent patients. as a member of the metastatic cancer task force, cdmrp encourages applicants to review the recommendations (https://health.mil/referencecenter/congressionaltestimonies/2018/05/03/metastaticcancer**research**) and submit **research** ideas to address these recommendations provided they are within the limitations of this funding opportunity and fit within the fy19 pcrp priorities. the proposed **research** must be relevant to active duty service members, veterans, military beneficiaries, and/or the american public. the anticipated direct costs budgeted for the entire period of performance for an fy19 idea development award will not exceed $600,000 for new investigators or $750,000 for established investigators. refer to section ii.d.5, funding restrictions, for detailed funding information. the cdmrp expects to allot approximately $35.52m to fund approximately 12 new investigator and 20 established investigator idea development award applications. funding of applications received is contingent upon the availability of federal funds for this program as well as the number of applications received, the quality and merit of the applications as evaluated by scientific and programmatic review, and the requirements of the government. funds to be obligated on any award resulting from this funding opportunity will be available for use for a limited time period based on the fiscal year of the funds. it is anticipated that awards made from this fy19 funding opportunity will be funded with fy19 funds, which will expire for use on september 30, 2025. awards will be made no later than september 30, 2020. for additional information, refer to section ii.f.1, federal award notices. the types of awards made under the program announcement will be assistance agreements (grants or cooperative agreements). the level of involvement on the part of the department of defense (dod) during project performance is the key factor in determining whether to award a grant or cooperative agreement. an assistance agreement (grant or cooperative agreement) is appropriate when the federal government transfers a &#8220;thing of value&#8221; to a &#8220;state, local government,&#8221; or &#8220;other recipient&#8221; to carry out a public purpose of support or stimulation authorized by a law of the united states, instead of acquiring property or service for the direct benefit and use of the u.s. government. an assistance agreement can take the form of a grant or cooperative agreement. if &#8220;no substantial involvement&#8221; on the part of the funding agency is anticipated, a grant award will be made (31 usc 6304). conversely, if substantial involvement on the part of the funding agency is anticipated, a cooperative agreement will be made (31 usc 6305) and the award will identify the specific substantial involvement. substantial involvement may include **collaboration**, participation, or intervention in the **research** to be performed under the award. the award type, along with the start date, will be determined during the negotiation process. clinical trials are not allowed. new fy19 definition: a clinical trial is a **research** study in which one or more human subjects are prospectively assigned to one or more interventions (which may include placebo or other control) to evaluate the effects of the interventions on biomedical or behavioral healthrelated outcomes. **research** involving human anatomical substances, human subjects, or human cadavers: all dodfunded **research** involving new and ongoing **research** with human anatomical substances, human subjects, or human cadavers must be reviewed and approved by the usamrmc orp, human **research** protection office (hrpo), prior to **research** implementation. this administrative review requirement is in addition to the local institutional review board (irb) or **ethics** committee (ec) review. local irb/ec approval at the time of submission is not required. the hrpo is mandated to comply with specific laws and requirements governing all **research** involving human anatomical substances, human subjects, or human cadavers that is supported by the dod. these laws and requirements will necessitate information in addition to that supplied to the irb/ec. allow a minimum of 2 to 3 months for hrpo regulatory review and approval processes. additional time for regulatory reviews may be needed for clinical studies taking place in international settings. when possible, protocols should be written for **research** with human subjects and/or human anatomical substances that are specific to the dodsupported effort outlined in the submitted application as a standalone study. submission to hrpo of protocols involving more than the scope of work in the dodfunded award will require hrpo review of the entire protocol (dod and nondod funded). dod human subjects protection requirements may be applied to nondod funded work and necessitate extensive revisions to the protocol. applications that involve recruitment of human subjects must indicate the quarterly enrollment targets across all sites in attachment 5: statement of work (sow). successful applicants will work with usamraa to establish milestones for human subjects recruitment. continued support for the project will be based upon satisfactory progress in meeting the established milestones. refer to the general application instructions, appendix 1, and the human subject resource document available on the electronic biomedical **research** application portal (ebrap) &#8220;funding opportunities &amp; forms&#8221; web page (https://ebrap.org/ebrap/public/program.htm) for additional information. use of dod or department of veterans affairs (va) resources: if the proposed **research** involves access to active duty military patient populations and/or dod resources or **database**s, the application must describe the access at the time of submission and include a plan for maintaining access as needed throughout the proposed **research**. access to target active duty military patient population(s) and/or dod resource(s) or **database**(s) should be confirmed by including a letter of support, signed by the lowestranking person with approval authority. if the proposed **research** involves access to va patient populations, va study resources and **database**s, and/or va **research** space and equipment, va principal investigators (pi)s/copis must have a plan for obtaining and maintaining access throughout the proposed **research**. access to va patients, resources, and/or va **research** space should be confirmed by including a letter of support from the va facility director(s) or individual designated by the va facility director(s), such as the associate chief of staff for **research** and development (acos/r&amp;d) or clinical service chief. if appropriate, the application should identify the vaaffiliated nonprofit corporation (npc) as the applicant institution for va pis. if the va npc is not identified as the applicant institution for administering the funds, the application should include a letter from the va acos/r&amp;d confirming this arrangement and identifying the institution that will administer the funds associated with the proposed **research**. access to certain dod or va patient populations, resources, or **database**s may only be obtained by **collaboration** with a dod or va investigator who has a substantial role in the **research** and may not be available to a nondod or nonva investigator if the resource is restricted to dod or va personnel. investigators should be aware of which resources are available to them if the proposed **research** involves a nondod or nonva investigator collaborating with the dod and/or va. if access cannot be confirmed at the time of application submission, the government reserves the right to withdraw or revoke funding until the pi has demonstrated support for and access to the relevant population(s) and/or resource(s). refer to section ii.d.2.b.ii, full application submission components, for detailed information. **research** involving animals: all dodfunded **research** involving new and ongoing **research** with animals must be reviewed and approved by the usamrmc orp animal care and use review office (acuro), in addition to the local institutional animal care and use committee (iacuc) of record. iacuc approval at the time of submission is not required. specific documents relating to the use of animals in the proposed **research** will be requested if the application is selected for funding. the acuro must review and approve all animal use prior to the start of working with animals, including amendments to ongoing projects. pis must submit the institutional animal use protocol, iacuc approval of that protocol, and a version of the animal use appendix titled, &#8220;**research** involving animals.&#8221; allow at least 2 to 3 months for acuro regulatory review and approval processes for animal studies. refer to the general application instructions, appendix 1, for additional information. all projects should adhere to a core set of standards for rigorous study design and reporting to maximize the **reproducibility** and translational potential of preclinical **research**. the standards are described in landis, s.c., et al., a call for transparent reporting to optimize the predictive value of preclinical **research**, nature 2012, 490:187191 (www.nature.com/nature/journal/ v490/n7419/full/nature11556.html). while these standards are written for preclinical studies, the basic principles of randomization, blinding, samplesize estimation, and data handling derive from wellestablished best practices in clinical studies. applicants should consult the arrive (animal **research**: reporting in vivo experiments) guidelines to ensure relevant aspects of rigorous animal **research** are adequately planned for and, ultimately, reported. the arrive guidelines can be found at https://www.elsevier.com/__data/promis_misc/ 622936arrive_guidelines.pdf. the cdmrp intends that information, data, and **research** resources generated under awards funded by this program announcement be made available to the **research** community (which includes both scientific and consumer advocacy communities) and to the public at large. for additional guidance, refer to the general application instructions, appendix 2, section k.

#### Evaluating the top match

In the above block, words matching keywords from our search list should be bolded.  There should be **ten** unique words bolded.

One thing to note right from the start is that there is a lot of text in this description.  Going back to the histogram we generated in the "Open Science Overview" chapter, this description is certianly in the right tail of that distribution with 750 words.  As such, simply due to there being more potential words to match upon, we might expect this description has a higher likelihood of meeting our inclusion criteria.

One other thing to note is that, although the word research shows up an _extreme_ number of times, it only counts as a single hit, so we needn't be concerned that resuse of a term is unfairly biasing our search algorithm results.

In any case, we can see that this isn't a grant which is _directly_ targeting open science.  Rather, it seems to be seeking submissions which are largely consistent with the values of open science.  In this way, this grant opportunity can be seen as an effort to fund open science (in that it seeks to fund the conduct of scientific investigations which are aligned with the principles of open science).  As such, it seems fairly reasonble that this grant would show up in our corpus.  That being said, we should also note that we shouldn't consider the _magnitude_ of the match (i.e. ten matching terms) as an indicator of how strong the grant's alignment is with the prinicples of open science.

Let's take a look at match two.

### Viewing the second top match

In [36]:
# get the description match for the current text
secondTopMatchText=grantsDF['Description'].loc[grantsDF['OpportunityID'].eq(grantIDs[topIndexes[-2]])].values[0]
# bold the words that match
for iKeyword in keywords:

    secondTopMatchText=secondTopMatchText.replace(iKeyword,'**'+iKeyword+'**')
# display the text as markdown
printmd(secondTopMatchText)

the geosciences open science ecosystem (geo ose) program seeks to support sustainable and networked open science activities to foster an ecosystem of inclusive access to data, physical collections, software, advanced computing, and other resources toward advancing **research** and education in the geosciences. the purpose of this support is to broadly enable geoscientists to leverage expanding information resources and computing capabilities to address interdisciplinary grand challenge **research** questions at the forefront of the geosciences. priority goals for geo ose are to: (i) improve the **openness** and scientific value of the existing network of cyberinfrastructure resources in the geosciences and related fields, such as data **repositories**, opensource software communities, and shared computing resources (e.g., highperformance and cloud computing), including via alignment on and adoption of common data and **metadata** standards that advance access and interoperability; (ii) democratize access to cyberinfrastructure capabilities that enable innovative geosciences **research** and education, including by advancing cloudbased approaches and **workflow**s; (iii) strengthen the capacity of current and future geoscientists to access, utilize, and collaborate within the growing ecosystem of open science resources; and (iv) contribute to advancing open science principles within the geosciences, including (but not limited to) the fair guiding principles for scientific **data management** and stewardship (findable, accessible, interoperable, reusable), the care principles for indigenous data governance (collective benefit, authority to control, responsibility, and **ethics**), and the trust principles for digital **repositories** (**transparency**, responsibility, user focus, sustainability, and technology), as well as **reproducibility** and **replicability**.

#### Evaluating the second top match 

In this example we find a much smaller body of text (240 words) but which still has 9 term matches.  Further more, we see that this description is spot-on for a direct and intentional open science-focused grant.  Indeed, it even uses the term "open science" directly.  This is precisely the kind of grant example we would hope to see.

### Viewing the fifth top match

Why are we viewing the fifth and not the third or fourth, you ask?  Because the text is remarkably similar to the first match, but with some small parts changed.  It seems that the DOD includes a great deal of boiler plate text in their grant submissions which covers the same topics. As such, much of what we said for that grant would apply here as well.

In [45]:
# get the description match for the current text
fifthTopMatchText=grantsDF['Description'].loc[grantsDF['OpportunityID'].eq(grantIDs[topIndexes[-5]])].values[0]
# bold the words that match
for iKeyword in keywords:

    fifthTopMatchText=secondTopMatchText.replace(iKeyword,'**'+iKeyword+'**')
# display the text as markdown
printmd(fifthTopMatchText)

notice of intent to award this funding announcement is not a request for applications. this announcement is to provide public notice of the national park service&#65533;&#65533;&#65533;s intention to fund the following project activities without full and open competition. abstract funding announcement p15as00214 project title collaborative biological ****research**** and education recipient california academy of science principle investigator / program manager scott loarie total anticipated award amount $140,000.00 cost share none new award or continuation? new cooperataive agreement anticipated length of agreement from date of award for five years anticipated period of performance from date of award for five years award instrument cooperative agreement statutory authority 54 usc 101702(a) cfda # and title 15.944 natural resources stewardship single source justification criteria cited unique qualifications nps point of contact june zastrow 3039876718 june_zastrow@nps.gov overview the objective of this agreement is to implement a program to engage in cooperating activities focused on promoting scientific ****research****, fostering a greater understanding of species distribution and the communities they form, and improving ecological inventory and education related to nps lands. parks have critical ****research**** and project management needs required for guiding management and policy, so that decisions are based on sound scientific information. furthermore, the nps lacks specific technical expertise and information technology systems to address pressing ****research**** and **data management** issues. examples of needs for ****research**** include analysis of nonnative species distribution, threatened and endangered species occurrences, phenological development of organisms, and changes in species distribution in the face of climate change. additionally, academy develops and maintains inaturalist, an interactive internet data platform already used by universities, agencies, and members of the public across the globe. integration of nps and existing academy data will enrich the value of a widely used ****research****, education, and data storage web portal. this partnership will allow nps and academy to collaboratively enrich ****research**** and education opportunities to the public. the nps is at a crossroads as it prepares for the bureau&#65533;&#65533;&#65533;s second century. nps is focused on a strategic integration of our national parks and communitybased programs. these efforts will better leverage our resources and engage the american public and citizen scientists to learn more about and contribute to our stewardship mission. through the go digital action, the nps digital experience will bring our online presence fully into the 21st century not only for accurate information, but for compelling knowledge and extended reach for communitybased programs. this agreement provides the terms and conditions under which the nps and academy will facilitate and cooperate in ****research**** and related activities of mutual benefit and of benefit to the people of the united states related to the protection and management of the natural resources of the nps. specifically, this agreement will facilitate nps and academy cooperation in completing ****research**** and education programs that protect, explain, and restore natural resources of the united states. the academy is a california nonprofit public benefit corporation, and is authorized to enter into agreements with the federal government. the academy is uniquely qualified to engage in community **collaboration**s with stakeholders and private funders to enhance the mission of the nps, including ****research**** and education. the academy is also uniquely qualified to facilitate citizen involvement and create partnerships for the common goals of the nps and community, including community representation in park initiatives. both nps and academy desire to cooperate with one another for their mutual benefit and for the general benefit of the people of the united states and future generations to enjoy the natural and cultural resources of the national park system. recipient involvement 1. collaboratively undertake a project tentatively entitled &#65533;&#65533;&#65533;collaborative biological ****research**** and education&#65533;&#65533;&#65533; (the &#65533;&#65533;&#65533;project&#65533;&#65533;&#65533;) involving collaborative natural resources ****research**** and **data sharing** projects and programs as further described throughout this article iii statement of work. 2. administer, facilitate and recruit interested students, organizations, agencies, or other individuals to participate in the collaborative natural resources ****research**** and **data sharing** programs under the project. 3. independently prepare application packages for ****research**** projects of mutual interest for scientific ****research****, inventory, and monitoring related to the objectives of the project as defined in article i of this agreement. 4. hire, assign and supervise scientists, technicians, and other support staff as needed to conduct ****research**** and education initiatives in any task agreements (tas) and to implement nps directed conservation projects and investigations in support of the project objectives. 5. provide the staff and their expertise and resources and to carry out the project objectives. 6. maintain functionality and capacity of existing inaturalist software which includes **open source** ios, android and web apps. 7. provide the nps with final reports and/or refereed journal articles (as agreed upon in advance and as defined in each ta), progress reports, and ******research**** data** (including **metadata**) generated by academy personnel from projects and programs conducted under the project. 8. provide personnel, facilities, supplies, materials and equipment as available and necessary to perform individual project tasks under the project. 9. collaborate with nps staff in supplying academy data and information tools to students in natural resource ****research****, monitoring, and restoration disciplines. 10. acknowledge the nps in any published or formally presented material developed or derived from this agreement. national park service involvement substantial involvement on the part the national park service is anticipated for the successful completion of the objectives to be funded by this award. in particular, the national park service will be responsible for the following: 1. collaboratively undertake the project as described throughout this article iii statement of work. 2. review and provide comment and/or concurrence to academy on plans, specifications and other documents submitted pursuant to article iii. a.1 above, within thirty (30) days of receipt. 3. provide academy with educational training regarding environmental and nps values. 4. assist with data collection and analysis on natural resources projects under the project. 5. provide project guidance and orientation, and provide necessary nps data for academy use. 6. participate in biodiversity ****research**** and education by providing park staff and expertise and **database** capabilities to collect and manage the information collected through this project. 7. provide a description of nps ****research**** and management needs through the appropriate planning documents and from direct contacts between nps staff and academy. 8. review all application packages submitted by academy and consider supporting the ****research**** contingent upon availability of funds and relation to ****research**** needs of mutual interest. proposals selected for funding shall be awarded under this agreement as individual ta&#65533;&#65533;&#65533;s. 9. initiate activities pursuant to the project under this agreement through the issuance of detailed ta&#65533;&#65533;&#65533;s. 10. provide access to and use of the natural resources of the nps for appropriate ****research****, monitoring, and educational activities of academy, except for those activities which may conflict with the values and purposes of the area as stated in federal law or policy. 11. provide necessary npsissued permits (e.g. collecting, special use, ****research****, etc.) pursuant to applicable law, and assist academy in securing other agency permits, as needed, for work conducted within the national parks under this agreement. 12. provide timely **peer review** of draft reports and technical publications that may be required. the academy and nps, jointly, agree to: 1. perform ****research****, monitoring, and educational programs consistent with the nature and intent of this agreement, and as described in individual ta&#65533;&#65533;&#65533;s placed under this agreement. 2. identify and obtain all necessary state and federal permits required to conduct the project. 3. consult the other before publishing any project results of joint effort. 4. be substantially involved in all phases of planning, ****research****, coordination, and publication and/or dissemination of final results. field activities can be considered to be joint ventures with the nps and academy contributing either primary direction and/or field support to accomplish project objectives. 5. recruit and direct the participation of graduate and undergraduate students, and interns. singlesource justification department of the interior single source policy requirements department of the interior policy (505 dm 2) requires a written justification which explains why competition is not practicable for each singlesource award. the justification must address one or more of the following criteria as well as discussion of the program legislative history, unique capabilities of the proposed recipient, and costsharing contribution offered by the proposed recipient, as applicable. in order for an assistance award to be made without competition, the award must satisfy one or more of the following criteria: (1) unsolicited proposal &#65533;&#65533;&#65533; the proposed award is the result of an unsolicited assistance application which represents a unique or innovative idea, method, or approach which is not the subject of a current or planned contract or assistance award, but which is deemed advantageous to the program objectives; (2) continuation &#65533;&#65533;&#65533; the activity to be funded is necessary to the satisfactory completion of, or is a continuation of an activity presently being funded, and for which competition would have a significant adverse effect on the continuity or completion of the activity; (3) legislative intent &#65533;&#65533;&#65533; the language in the applicable authorizing legislation or legislative history clearly indicates congress&#65533;&#65533;&#65533; intent to restrict the award to a particular recipient of purpose; (4) unique qualifications &#65533;&#65533;&#65533; the applicant is uniquely qualified to perform the activity based upon a variety of demonstrable factors such as location, property ownership, voluntary support capacity, costsharing ability if applicable, technical expertise, or other such unique qualifications; (5) emergencies &#65533;&#65533;&#65533; program/award where there is insufficient time available (due to a compelling and unusual urgency, or substantial danger to health or safety) for adequate competitive procedures to be followed. the national park service did not solicit full and open competition for this award based the following criteria: (4) unique qualifications single source justification description: the applicant is uniquely qualified to perform the activity based upon a variety of demonstrable factors such as location, property ownership, voluntary support capacity, costsharing ability, if applicable, technical expertise, or other such unique qualifications; &#65533;&#65533;&#65533; natural history museum organization: academy&#65533;&#65533;&#65533;s mission is to explore, explain, and sustain life. academy curates one of the world&#65533;&#65533;&#65533;s largest specimen collections of biodiversity. &#65533;&#65533;&#65533; relevant experience: academy&#65533;&#65533;&#65533;s inaturalist has been adopted as a wildlife recording platform by many organizations including parks canada, the mexican government&#65533;&#65533;&#65533;s conabio, and national geographic. inaturalist was used as the platform in the 2014 world parks congress bioblitz, the 2015 **citizen science** association bioblitz, the 2014 golden gate national parks bioblitz, and will be used in the upcoming 2015 hawaii volcanoes national park bioblitz and national geographic global snapshot. cumulatively, the academy through inaturalist stored information provides a continental scope of biodiversity information. &#65533;&#65533;&#65533; large existing user community: academy&#65533;&#65533;&#65533;s inaturalist program is supported by a vibrant user community of nearly 80,000 naturalists, scientists, managers, and members of the public that bring the benefits of crowdsourced species identification, curation of taxonomy, and sources of educational content to new **collaboration**s that leverage this network. over 1,180,000 wildlife observations from around the world representing over 60,000 distinct taxa are being shared and, to date, over 610,000 inaturalist &#65533;&#65533;&#65533;****research**** quality&#65533;&#65533;&#65533; observations have been archived by the global biodiversity information facility. &#65533;&#65533;&#65533; collaborative effort: inaturalist is an **open source** software effort and development of the platform has been aided by financial and development support by numerous collaborators such as the mexican government&#65533;&#65533;&#65533;s conabio, spain&#65533;&#65533;&#65533;s creaf organization, and the new zealand biological recording network. &#65533;&#65533;&#65533; taxonomic scope: academy&#65533;&#65533;&#65533;s inaturalist program spans the tree of life. &#65533;&#65533;&#65533; geographic scope: academy&#65533;&#65533;&#65533;s inaturalist program spans the united states land area. inclusion of all category ii protected areas listed by the international union for conservation of nature (iucn) in canada and the world. &#65533;&#65533;&#65533; treatment of sensitive species locations: academy&#65533;&#65533;&#65533;s inaturalist includes all conservation statuses from the iucn redlist of threatened species and has tools for obscuring the sensitive locations of threatened species from the public. &#65533;&#65533;&#65533; ability to group and visualize observations against protected area boundaries: academy&#65533;&#65533;&#65533;s inaturalist **database** includes global information system functionality including the ability to filter, analyze, and visualize point observations within polygons such as protected area boundaries. &#65533;&#65533;&#65533; availability of platform on ios and android devices: academy&#65533;&#65533;&#65533;s inaturalist is available as a web platform and also as native ios and android applications freely available from the itunes and google play stores respectively. &#65533;&#65533;&#65533; science tool  availability of data for incorporation into scientific **database**s: academy&#65533;&#65533;&#65533;s inaturalist program generates ****research**** quality data which have been incorporated into the global biodiversity information facility (gbif), encyclopedia of life, and natureserve kestrel scientific **database**s. over 92% of digitized photos contributed to gbif dated in the last five years have come from inaturalist. &#65533;&#65533;&#65533; **open source** software and licensed data promoting public access: academy&#65533;&#65533;&#65533;s inaturalist program is powered by **open source** software freely available for use by the public. statutory authority 54 usc 101702(a) previously 16 u.s.c. &#65533;&#65533;1g cooperative agreements, transfer of service appropriated funds

#### Evaluating the fifth top match 

Here we see an example of a program directly soliciting technical capacties and expertise to augment existing organizations.  With respect to our open science investigation there are several key things to note.  One is the statement "Furthermore, the nps lacks specific technical expertise and information technology systems to address pressing research and data management issues".  In this statement we find a fascinating and candid acknowledgment about the National Park Service's current lack of capacity to meet the demands of open science.  Later, the description provides an overview of the citizen science and participant involvment in the associated projects, noting the broad involvment of a great many individuals--a sure hallmark of a a desired open science outcome.

### Checking a non match

